In [7]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
a = []

f = open('20220511T174759.log')
line = f.readline()
while line:
    line = f.readline()
    a.append(line)
    
f.close()    

In [6]:
a[0]

'11052022:160119;cherepovets;ak1456_1881;#;bus;59.1339225769043;37.97321701049805;0.00;0;0;;е254хк_35 [БВ (Н)];\n'

In [13]:
df = pd.DataFrame([sub.split(":") for sub in a])
df.columns = ['date', 'data']

In [25]:
df_append = []

In [26]:
df_t = pd.DataFrame(df.data[0].split(";")).T

In [27]:
df_append.append(df_t)

In [33]:
df_append = []
for i in range(len(df.data)):
    try:
        df_append.append(pd.DataFrame(df.data[i].split(";")).T)
    except AttributeError:
        pass

In [34]:
pd.concat(df_append)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,160119,cherepovets,ak1456_1881,#,bus,59.1339225769043,37.97321701049805,0.00,0,0,,е254хк_35 [БВ (Н)],\n
0,160117,cherepovets,ak1456_2424,#7,bus,59.12198257446289,37.97989273071289,9.44,8,1,,е401хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
0,160117,cherepovets,ak1456_1946,#27,bus,59.096702575683594,37.9358024597168,28.49,280,1,,е387хк_35 [БВ (Н)],27 [ул. монт - клер - > - доменная]\n
0,160118,cherepovets,ak1456_1892,#8,bus,59.126468658447266,37.91294479370117,17.76,103,1,,е329хк_35 [БВ (Н)],8 [доменная - > - ул. красная]\n
0,160115,cherepovets,ak1456_2408,#18,bus,59.1187744140625,37.90309143066406,0.00,13,1,,е407хк_35 [БВ (Н)],18 [ул. рыбинская (2) -> пр. победы 2]\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,174756,cherepovets,novotrans_1146909,#37,bus,59.124255,37.9060116667,9.00,103,1,,,37 зима*_февраль Доменная — Ветеранов\n
0,174750,cherepovets,novotrans_200040110,#25,bus,59.1014066667,37.908485,14.00,145,1,,,25*_ФЕВРАЛЬ Доменная — Рыбинская\n
0,174754,cherepovets,novotrans_14493723,#38,bus,59.1268466,37.9565166,31.00,282,1,,,38_февраль ГОГОЛЯ - ДОМЕННАЯ\n
0,174754,cherepovets,novotrans_14867608,#,bus,59.136538,37.977195,67.00,164,0,,,\n
